In [142]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import logging
import secrets
stock_list_loc = 'stocks_list.csv'
host = 'localhost'
tokens_lifetime_sec = 300

In [173]:
def init_db(reinit = False):
    con = psycopg2.connect(host =host, user='postgres', password='postgres');
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = con.cursor();
    if reinit:
        logging.info('droping db')
        cursor.execute("DROP DATABASE IF EXISTS stocks;")
    cursor.execute("SELECT datname FROM pg_database WHERE datname = %s;", ('stocks',))
    if len(cursor.fetchall()) == 0:
        sqlCreateDB = f"create database stocks;"
        cursor.execute(sqlCreateDB)
    cursor.close();
    con.close()    
    
    
    con = psycopg2.connect(f"dbname=stocks user=postgres password='postgres'");
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = con.cursor();
    logging.info('creating tables')
    sqlCreateTableUser = "CREATE TABLE IF NOT EXISTS users(nick VARCHAR(256) PRIMARY KEY, hash VARCHAR(512) NOT NULL, token VARCHAR(512) NOT NULL, token_date timestamp NOT NULL);"
    sqlCreateTableStock = "CREATE TABLE IF NOT EXISTS stocks(code VARCHAR(10) PRIMARY KEY);"
    sqlCreateTableUserStock = '''CREATE TABLE IF NOT EXISTS
    user_stock (
    id serial PRIMARY KEY,
    nick VARCHAR (256) NOT NULL,
    code VARCHAR (10) NOT NULL,
    FOREIGN KEY (nick) REFERENCES users (nick),
    FOREIGN KEY (code) REFERENCES stocks (code),
    UNIQUE (nick, code)
    );
    '''
    for query in [sqlCreateTableUser, sqlCreateTableStock, sqlCreateTableUserStock]:
        cursor.execute(query)
    cursor.execute('select COUNT(*) from stocks')
    if cursor.fetchone()[0] == 0:
        logging.info('inserting stocks')
        with open(stock_list_loc, 'r') as f:
            for s in f:
                cursor.execute(f"INSERT INTO stocks VALUES ('{s.strip()}')")
    else:
        logging.info('stocks already presented')
    cursor.close()
    con.close()    

In [174]:
init_db(True)

In [177]:
con = psycopg2.connect(f"dbname=stocks user=postgres password='postgres'");
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = con.cursor();

In [163]:
def get_all_stocks():
    cursor.execute('SELECT * from stocks')
    for x in cursor.fetchall():
        yield x[0]

In [168]:
def get_user_stocks(user):
    cursor.execute(f"SELECT code from user_stock WHERE nick = '{user}'")
    for x in cursor.fetchall():
        yield x[0]

In [159]:
def add_stock_to_user(user, stock):
    stock = stock.upper()
    cursor.execute(f"INSERT INTO user_stock(nick, code) VALUES ('{user}','{stock}')")

In [113]:
def remove_stock_from_user(user, stock):
    stock = stock.upper()
    cursor.execute(f"DELETE FROM user_stock WHERE nick='{user} AND code='{stock}'")

In [125]:
def register_user(user, passwd):
    cursor.execute(f"SELECT COUNT(*) FROM users WHERE nick='{user}'")
    if cursor.fetchone()[0] != 0:
        print('alarm')
        return
    passwd = hash(passwd)
    token = secrets.token_hex(64)
    cursor.execute(f"INSERT INTO users VALUES ('{user}', '{passwd}', '{token}', Now())")
    logging.info(f'registered {user}')
    return token

In [179]:
#alarm
def check_token(token):
    cursor.execute(f"SELECT now() - token_date FROM users WHERE token='{token}'")
    res = cursor.fetchall()
    if len(res) == 0:
        return 'no such token'
    res = res[0]
    left_time = res[0]
    if left_time.seconds < tokens_lifetime_sec:
        return 'ok'
    return 'outdated'

In [134]:
def get_token(user, passwd):
    passwd = hash(passwd)
    cursor.execute(f"SELECT COUNT(*) FROM users WHERE nick='{user}' and hash='{passwd}'")
    if cursor.fetchone()[0] == 0:
        return 'nope'
    token = secrets.token_hex(64)
    cursor.execute(f'''UPDATE users
    SET token = '{token}',
    token_date = now()
    WHERE nick = '{user}'
    ''')
    return token

In [183]:
check_token('5a82823879110b314574ed4fd49e35402f9bfd8bd1d46a6daeba652826dcab8de2b3375e7e3743f296c892b4c7f8aa8631f47870eafd2e3337a922ae413b88df')

'ok'

In [181]:
get_token('sergeyer', '123')

'nope'

In [182]:
register_user('sergeyer', '123')

registered


'5a82823879110b314574ed4fd49e35402f9bfd8bd1d46a6daeba652826dcab8de2b3375e7e3743f296c892b4c7f8aa8631f47870eafd2e3337a922ae413b88df'

In [169]:
add_stock_to_user('sergeyer', 'FB')

In [170]:
for x in get_user_stocks('sergeyer'):
    # обработать ошибки, пересоздавать подключение
    print(x)

PFE
FB


In [104]:
for i in get_all_stocks():
    print(i)

BDN
ROK
ZM
CRM
AGCO
TSLA
MRNA
AAPL
FB
GOOGL
PFE
GM
GE
F


In [171]:
cursor.close()
con.close()

In [167]:
cursor.execute(f"SELECT code from user_stock WHERE nick = 'sergeyer'")
for x in cursor.fetchall():
    print(x)

('PFE',)
